In [14]:
import numpy as np
from gym import spaces
from dataprep.dataset import load_reformated_csv, create_dataset
import json
import keras
from globals import *
import pandas as pd

In [32]:
def get_dataset(df ,
                variable: str = 'B:VIMIN' ,
                train_test_split: float = 0.7) :
    dataset = df[variable].values
    dataset = dataset.astype( 'float32' )
    dataset = np.reshape( dataset , (-1 , 1) )

    train_size = int( len( dataset ) * train_test_split )
    train , test = dataset[0 :train_size , :] , dataset[train_size :len( dataset ) , :]

    X_train , Y_train = create_dataset( train , look_back = LOOK_BACK )
    X_train = np.reshape ( X_train , (X_train.shape [ 0 ] , X_train.shape [ 1 ], 1) )
    Y_train = np.reshape ( Y_train , (Y_train.shape [ 0 ] , Y_train.shape [ 1 ]) )

    return X_train , Y_train

def all_inplace_scale(df) :
    scale_dict = {}

    for var in VARIABLES :
        our_data2 = df
        trace = our_data2[var].astype( 'float32' )
        data = np.array( trace )

        median = np.median( data )
        upper_quartile = np.percentile( data , 75 )
        lower_quartile = np.percentile( data , 25 )

        iqr = upper_quartile - lower_quartile
        lower_whisker = data[data >= lower_quartile - 1.5 * iqr].min( )
        upper_whisker = data[data <= upper_quartile + 1.5 * iqr].max( )

        ranged = upper_whisker - lower_whisker
        # (value − median) / (upper - lower)
        our_data2[var] = 1 / ranged * (data - median)

        scale_dict[str( var )] = {"median" : median , "range" : ranged}

    return scale_dict

def unscale(var_name , tseries , scale_dict) :
    # equivalent to inverse transform
    from_model = np.asarray( tseries )
    update = from_model * scale_dict[str( var_name )]["range"] + scale_dict[str( var_name )]["median"]

    return (update)

def rescale(var_name , tseries , scale_dict) :
    # equivalent to transform
    data = np.asarray( tseries )
    update = 1 / scale_dict[str( var_name )]["range"] * (data - scale_dict[str( var_name )]["median"])

    return (update)

In [33]:
def create_dropout_predict_model(model , dropout) :
    # Load the config of the original model
    conf = model.get_config( )

    # Add the specified dropout to all layers
    for layer in conf['layers'] :
        if layer["class_name"] == "Dropout" :
            # print(layer)
            layer["config"]["rate"] = dropout
    model_dropout = keras.Model.from_config( conf )
    model_dropout.set_weights( model.get_weights( ) )
    return model_dropout


def regulation(alpha , gamma , error , min_set , beta) :
    ## calculate the prediction with current regulation rules
    ## from Rachael's report, eq (1)
    # beta=[0]
    ER = error  # error
    _MIN = min_set  # setting
    for i in range( len( _MIN ) ) :
        if i > 0 :
            beta_t = beta[-1] + gamma * ER[i]
            beta.append( beta_t )  # hopefully this will update self.rachael_beta in place

    MIN_pred = _MIN - alpha * ER - np.asarray( beta[-LOOK_BACK :] ).reshape( LOOK_BACK ,
                                                                             1 )  # predict the next, shiftting happens in the plotting #check here
    return MIN_pred

In [34]:
model = keras.models.load_model(
    filepath = LATEST_SURROGATE_MODEL ,
    compile = False )
booster_model = create_dropout_predict_model( model , .2 )

In [35]:
# Load data from config
with open( DATA_CONFIG ) as json_file :
    data_config = json.load( json_file )
data = load_reformated_csv( filename = data_config['data_dir'] + data_config['data_filename'] ,
                            nrows = NSTEPS )
scale_dict = all_inplace_scale( data )
data['B:VIMIN'] = data['B:VIMIN'].shift( -1 )
data = data.set_index( pd.to_datetime( data.time ) )
data = data.dropna( )
data = data.drop_duplicates( )
variables = VARIABLES
nvariables = len( variables )

data_list = []
x_train = []
# get_dataset also normalizes the data
for v in range( len( variables ) ) :
    data_list.append( get_dataset( data , variable = variables[v] ) )
    # self.scalers.append(data_list[v][0])
    x_train.append( data_list[v][0] )

In [36]:
concate_axis = 2
X_train = np.concatenate(x_train, axis=concate_axis)
print( 'Data shape:{}'.format( X_train.shape ) )
nbatches = X_train.shape[0]
nsamples = X_train.shape[1]

Data shape:(174982, 15, 6)


In [37]:
state = np.zeros(shape=(1, 15, 6))
predicted_state = np.zeros(shape = (1, 6, 1))
rachael_state = np.zeros(shape=(1, 15, 6))
rachael_predicted_state = np.zeros(shape = (1, 6, 1))

In [38]:
from gym import spaces
observation_space = spaces.Box(
    low = 0 ,
    high = +1 ,
    shape = (6 ,) ,
    dtype = np.float64
    )
actionMap_VIMIN = [0 , 0.0001 , 0.005 , 0.001 , -0.0001 , -0.005 , -0.001]
action_space = spaces.Discrete( 7 )
VIMIN = 0
action = 4

In [39]:
delta_VIMIN = actionMap_VIMIN[action]
DENORN_BVIMIN = unscale( variables[0] , np.array( [VIMIN] ).reshape( 1 , -1 ) ,
                                 scale_dict )
DENORN_BVIMIN += delta_VIMIN
print(DENORN_BVIMIN)

[[103.39762797]]


In [41]:
alpha = 10e-2
gamma = 7.535e-5
rachael_beta = [0]
B_VIMIN_trace = unscale( variables[2] , state[0 , : , 2].reshape( -1 , 1 ) ,
                         scale_dict )
BIMINER_trace = unscale( variables[1] , state[0 , : , 1].reshape( -1 , 1 ) ,
                         scale_dict )

rachael_state[0][nsamples - 1][0] = rescale( variables[0] ,
                                           regulation( alpha , gamma ,
                                                       error = BIMINER_trace ,
                                                       min_set = B_VIMIN_trace ,
                                                       beta = rachael_beta )[
                                               -1].reshape( -1 , 1 ) ,
                                           scale_dict ) 

In [47]:
VIMIN = rescale( variables[0] , DENORN_BVIMIN ,
                  scale_dict ) 
state[0][nsamples - 1][0] = VIMIN
state

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.     

In [51]:
predicted_state = booster_model.predict( state ).reshape(1, 2, 1)
predicted_state

array([[[ 1.8397697],
        [-1.567316 ]]], dtype=float32)

In [52]:
state[0 , 0 :-1, :] = state[0, 1 :, :]  # shift forward
rachael_state[0 , 0 :-1, : ] = rachael_state[0 , 1 :, : ]

# Update IMINER
state[0][nsamples - 1][1] = predicted_state[0 , 1 :2]
rachael_state[0][nsamples - 1][1] = rachael_predicted_state[0 , 1 :2]

In [57]:
data_state = None
data_state = np.copy(
    X_train[1000 + 1].reshape( 1 , 15, 6 ) )
data_iminer = unscale( variables[1] ,
                       data_state[0][nsamples - 1][1].reshape( 1 , -1 ) ,
                       scale_dict )


array([[0.0746875]], dtype=float32)

In [60]:
state[0 , :, 2 :6] = data_state[0 , :, 2 :6]
rachael_state[0 , :, 2:6] = data_state[0 , :, 2 :6]

In [64]:
state[0, -1 :, :].flatten( )

(6,)

In [65]:
batch_id = np.random.randint( low = 500 , high = 5000 )

In [72]:
state = np.copy(X_train[batch_id].reshape(1, 15, 6))
state[0 , -1:, :].flatten( ).shape

(6,)